## Bonus: Poruchy


In [79]:
import pandas
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC


Stáhni si data o poruchách ze souboru [predictive_maintenance.csv](predictive_maintenance.csv). Význam sloupců je následující:

- UID: jedinečný identifikátor v rozsahu 1 až 10000,
- ID produktu: skládá se z písmene L, M nebo H pro nízkou (50 % všech výrobků), střední (30 %) a vysokou (20 %) variantu kvality výrobku a sériového čísla specifického pro danou variantu,
- teplota vzduchu (K),
- teplota procesu (K),
- otáčky (ot/min),
- točivý moment (Nm),
- opotřebení nástroje (min),
- označení "selhání stroje" (pokud k němu došlo).


In [80]:
data = pandas.read_csv("predictive_maintenance.csv")
data.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure



Tvým úkolem je vytvořit model, který bude predikovat poruchu stroje.

Proveď následující postup:

- Vyřaď z datasetu sloupce (jsou dva), které nemají pro analýzu význam.


In [81]:
data = data.drop(columns=["UDI", "Product ID"])

- Podívej se, kolik typů poruch bylo objeveno. Vyřaď nejméně častou poruchu.


In [82]:
data.groupby("Failure Type").size().sort_values(ascending=False)

Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
dtype: int64

In [83]:
data = data[data["Failure Type"] != "Random Failures"]

- Rozděl data na vstupní proměnné a výstupní proměnnou.


In [84]:
X = data.drop(columns=["Failure Type"])
y = data["Failure Type"]

In [85]:
X_type = pandas.get_dummies(X["Type"])
X = pandas.merge(X, X_type, left_index=True, right_index=True)
X = X.drop(columns=["Type"])
X

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H,L,M
0,298.1,308.6,1551,42.8,0,0,False,False,True
1,298.2,308.7,1408,46.3,3,0,False,True,False
2,298.1,308.5,1498,49.4,5,0,False,True,False
3,298.2,308.6,1433,39.5,7,0,False,True,False
4,298.2,308.7,1408,40.0,9,0,False,True,False
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,False,False,True
9996,298.9,308.4,1632,31.8,17,0,True,False,False
9997,299.0,308.6,1645,33.4,22,0,False,False,True
9998,299.0,308.7,1408,48.5,25,0,True,False,False


In [86]:
# encoder = OneHotEncoder()
# X = encoder.fit_transform(X)

# Tento řádek je pouze pro kontrolu
# pandas.DataFrame(X.toarray(), columns=encoder.get_feature_names_out()).head()

- Vyzkoušej algoritmy Support Vector Machine a K Nearest Neighbors ke klasifikaci poruchy stroje. 

Porovnej, který algoritmus dosáhl lepších výsledků.

In [87]:
model_1 = SVC()
params_1 = {"kernel": ["linear", "rbf"], "decision_function_shape": ["ovo", "ovr"]}

clf_1 = GridSearchCV(model_1, params_1, scoring="accuracy")
clf_1.fit(X, y)

print(clf_1.best_params_)
print(round(clf_1.best_score_, 2))

{'decision_function_shape': 'ovo', 'kernel': 'linear'}
1.0


In [88]:
model_2 = KNeighborsClassifier()
params_2 = {"n_neighbors": [3, 5, 11, 15]}

clf_2 = GridSearchCV(model_2, params_2, scoring="accuracy")
clf_2.fit(X, y)

print(clf_2.best_params_)
print(round(clf_2.best_score_, 2))

{'n_neighbors': 15}
0.97
